<a rel="license" href="http://creativecommons.org/licenses/by-nc-nd/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-nd/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-nd/4.0/">Creative Commons Attribution-NonCommercial-NoDerivatives 4.0 International License</a>.

![Spark Logo](http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png) ![Python Logo](http://spark-mooc.github.io/web-assets/images/python-logo-master-v3-TM-flattened_small.png)
# **Web Server Log Analysis with Apache Spark**

## Why Server Log Analysis is a suitable Big Data Problem
* ### Very large and common data source
* ### Sensitive information
* ### Requires repeated and quick corrective action

## Exploratory Data Analysis on the web log data

Let's begin looking at our data.  For this lab, we will use a data set from NASA Kennedy Space Center web server in Florida. The full data set is freely available at <http://ita.ee.lbl.gov/html/contrib/NASA-HTTP.html>, and it contains all HTTP requests for two months. We are using a subset that only contains several days' worth of requests.  The log file has already been downloaded for you.

In [ ]:
import sys
import os

log_file_path = '../files/NASA_access_log_Aug95'

### Loading the log file

Now that we have the path to the file, let's load it into a DataFrame. We'll do this in steps. First, we'll use `sqlContext.read.text()` to read the text file. This will produce a DataFrame with a single string column called `value`.

In [ ]:
base_df = sqlContext.read.text(log_file_path)
# Let's look at the schema
base_df.printSchema()

Let's take a look at some of the data.

In [ ]:
base_df.show(truncate=False)

### Parsing the log file

This is in [Common Log Format](https://www.w3.org/Daemon/User/Config/Logging.html#common-logfile-format). The fields are:


| field         | meaning                                                                |
| ------------- | ---------------------------------------------------------------------- |
| _remotehost_  | Remote hostname (or IP number if DNS hostname is not available).       |
| _rfc931_      | The remote logname of the user. We don't really care about this field. |
| _authuser_    | The username of the remote user, as authenticated by the HTTP server.  |
| _[date]_      | The date and time of the request.                                      |
| _"request"_   | The request, exactly as it came from the browser or client.            |
| _status_      | The HTTP status code the server sent back to the client.               |
| _bytes_       | The number of bytes (`Content-Length`) transferred to the client.      |


How to extract these fields from the each record?

Use the built-in [regexp\_extract()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.regexp_extract)
function to do the parsing. This function matches a column against a regular expression with one or more [capture groups](http://regexone.com/lesson/capturing_groups) and allows you to extract the match.

We will use one regex for each field.

In [ ]:
from pyspark.sql.functions import split, regexp_extract
split_df = base_df.select(regexp_extract('value', r'^([^\s]+\s)', 1).alias('host'),
                          regexp_extract('value', r'^.*\[(\d\d/\w{3}/\d{4}:\d{2}:\d{2}:\d{2} -\d{4})]', 1).alias('timestamp'),
                          regexp_extract('value', r'^.*"\w+\s+([^\s]+)\s+HTTP.*"', 1).alias('path'),
                          regexp_extract('value', r'^.*"\s+([^\s]+)', 1).cast('integer').alias('status'),
                          regexp_extract('value', r'^.*\s+(\d+)$', 1).cast('integer').alias('content_size'))
split_df.show()

### Data Cleaning - Did the parsing logic work?
#### How many null rows are there in the dataframe?

In [ ]:
base_df.filter(base_df['value'].isNull()).count()

#### How many rows with null columns?

In [ ]:
bad_rows_df = split_df.filter(split_df['host'].isNull() |
                              split_df['timestamp'].isNull() |
                              split_df['path'].isNull() |
                              split_df['status'].isNull() |
                             split_df['content_size'].isNull())
bad_rows_df.count()

#### Which columns have null values?

In [ ]:
from pyspark.sql.functions import col, sum

def count_null(col_name):
  return sum(col(col_name).isNull().cast('integer')).alias(col_name)

# Build up a list of column expressions, one per column.
#
# This could be done in one line with a Python list comprehension, but we're keeping
# it simple for those who don't know Python very well.
exprs = []
for col_name in split_df.columns:
  exprs.append(count_null(col_name))

# Run the aggregation. The *exprs converts the list of expressions into
# variable function arguments.
split_df.agg(*exprs).show()

#### Q: What went wrong?

In [ ]:
bad_content_size_df = base_df.filter(~ base_df['value'].rlike(r'\d+$'))
bad_content_size_df.count()

#### If these values don't end in digits, then what are the trailing characters?

In [ ]:
bad_content_size_df.show(10, truncate=False)

#### Q: What does the `-` mean and how to fix it?

### Fix the rows with invalid `content_size`

* [fillna()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.fillna), which fills null values with specified non-null values.
* [na](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.na), which returns a [DataFrameNaFunctions](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameNaFunctions) object with many functions for operating on null columns.

In [ ]:
# Replace all null content_size values with 0.
cleaned_df = split_df.na.fill({'content_size': 0})

In [ ]:
# Ensure that there are no nulls left.
exprs = []
for col_name in cleaned_df.columns:
  exprs.append(count_null(col_name))

cleaned_df.agg(*exprs).show()

### Parsing the timestamp.

In [ ]:
from pyspark.sql.functions import udf

In [ ]:
month_map = {
  'Jan': 1, 'Feb': 2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7,
  'Aug':8,  'Sep': 9, 'Oct':10, 'Nov': 11, 'Dec': 12
}

def parse_clf_time(s):
    """ Convert Common Log time format into a Python datetime object
    Args:
        s (str): date and time in Apache time format [dd/mmm/yyyy:hh:mm:ss (+/-)zzzz]
    Returns:
        a string suitable for passing to CAST('timestamp')
    """
    # NOTE: We're ignoring time zone here. In a production application, you'd want to handle that.
    return "{0:04d}-{1:02d}-{2:02d} {3:02d}:{4:02d}:{5:02d}".format(
      int(s[7:11]),
      month_map[s[3:6]],
      int(s[0:2]),
      int(s[12:14]),
      int(s[15:17]),
      int(s[18:20])
    )

u_parse_time = udf(parse_clf_time)

logs_df = cleaned_df.select('*', u_parse_time(cleaned_df['timestamp']).cast('timestamp').alias('time')).drop('timestamp')
total_log_entries = logs_df.count()

In [ ]:
logs_df.printSchema()

In [ ]:
logs_df.limit(10).toPandas().head()

In [ ]:
logs_df.cache()

## Analysis Walk-Through on the Web Server Log File

### Example: Content Size Statistics

We can compute the statistics by calling `.describe()` on the `content_size` column of `logs_df`.  The `.describe()` function returns the count, mean, stddev, min, and max of a given column.

In [ ]:
# Calculate statistics based on the content size.
content_size_summary_df = logs_df.describe(['content_size'])
content_size_summary_df.show()

### Exercise: Use the `sql_funcs.{count, mean, stddev, min, max}` functions to verify the summary statistics

In [ ]:
from pyspark.sql import functions as sql_funcs

In [ ]:
# enter code here

### Example: HTTP Status Analysis

In [ ]:
status_to_count_df =(logs_df
                     .groupBy('status')
                     .count()
                     .sort('status')
                     .cache())

status_to_count_length = status_to_count_df.count()
print('Found %d response codes' % status_to_count_length)
status_to_count_df.show()

### Example: Plotting the status counts

In [ ]:
%matplotlib inline

In [ ]:
status_to_count_df.limit(10).toPandas().set_index('status', drop=True).plot(kind='bar')

### Q: What is wrong with this graph?

In [ ]:
log_status_to_count_df = status_to_count_df.withColumn('log(count)',
                                                       sql_funcs.log(status_to_count_df['count']))

log_status_to_count_df.select('status', 'log(count)').toPandas().set_index('status', drop=True).plot(kind='bar')

### Exercise: Find the top 20 most frequent hosts from the database.

In [ ]:
# enter code here

###  Analyzing Web Server Log File

### Exercise: Top Ten Error Paths
#### Hint: An error is any HTTP request that does not have a return status of 200.
#### Hint: Use the `filter` operation, followed by grouping and sorting

In [ ]:
# enter code here

#### Exercise: Number of Unique Hosts

In [ ]:
# enter code here

## Exploring 404 Status Codes

404 errors are returned when the server cannot find the resource (page or object) the browser or client requested.

### Exercise: Counting 404 Response Codes

How many 404 records are in the log?

In [ ]:
# enter code here

### Exercise: List the Top Twenty 404 Response Code paths

In [ ]:
# enter code here